In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import lensing
import tools
import dipole_estimator
import stats
from tqdm import tqdm
import cosmo 
import ilc

In [3]:
# setting map parameters: number of pixels in x/y-direction, pixel size in x/y-direction in arcmin
mapparams = [240, 0.25, 240, 0.25] 


# ... 
nber_clusters = 1500
nber_random = 15000
nber_cov = 100


# ...
cluster_uncorr_fg = False
cluster = [6e14, 3., 1.]
cluster_corr_fg = False
noise = False
beam_fwhm = 1.4
noise_val = 2.0


# ...
correct_for_tsz = False


# ...
mass_int = np.arange(4e14, 8e14, 0.2e14)

In [ ]:
covariance_matrix, correlation_matrix = dipole_estimator.covariance(mapparams, nber_clusters, nber_random, nber_cov, beam_fwhm = beam_fwhm, noise_val = noise_val)

In [ ]:
dipole_profile_models = dipole_estimator.get_fit_profiles(mapparams, mass_int, cluster[1], cluster[2], beam_fwhm, noise_val)   

In [ ]:
plt.imshow(covariance_matrix, origin = 'lower')
plt.colorbar()

In [ ]:
sims = dipole_estimator.mock_data(mapparams, nber_clusters, cluster = cluster, cluster_corr_fg = cluster_corr_fg, cluster_uncorr_fg = cluster_uncorr_fg, beam_fwhm = beam_fwhm, noise_val = noise_val)
lensed_stack = dipole_estimator.get_stack(sims, mapparams, nber_clusters, noise_val) 
sims = dipole_estimator.mock_data(mapparams, nber_random, cluster = None, cluster_uncorr_fg = cluster_uncorr_fg, cluster_corr_fg = cluster_corr_fg, beam_fwhm = beam_fwhm, noise_val = noise_val)
unlensed_stack = dipole_estimator.get_stack(sims, mapparams, nber_random, noise_val)
bins, kappa_profile = dipole_estimator.get_dipole_profile(mapparams, lensed_stack, unlensed_stack)
#data = bins, kappa_profile, covariance_matrix
#params = stats.run_mcmc(data, dipole_profile_models, mass_int)

/Users/kevinlevy/Documents/codes/master_thesis/code/lensing.py:106: RuntimeWarning: divide by zero encountered in arctanh
  f[x3] = (1/np.sqrt(1-x[x3]**2))*np.arctanh(np.sqrt(1-x[x3]**2))
/Users/kevinlevy/Documents/codes/master_thesis/code/lensing.py:41: RuntimeWarning: invalid value encountered in true_divide
  alphaX_fft =  1j * lx * 2. *  kappa_fft / el2d**2
/Users/kevinlevy/Documents/codes/master_thesis/code/lensing.py:42: RuntimeWarning: invalid value encountered in true_divide
  alphaY_fft =  1j * ly * 2. *  kappa_fft  /el2d**2
/Users/kevinlevy/Documents/codes/master_thesis/code/dipole_estimator.py:18: RuntimeWarning: invalid value encountered in true_divide
  gradient_filter = gradient_filter/gradient_filter


In [ ]:
extent, grid = tools.make_grid(mapparams)
plt.subplots()
plt.imshow(lensed_stack, extent = extent)
plt.colorbar()
plt.title(r'Lensed Stack')
plt.subplots()
plt.imshow(unlensed_stack, extent = extent)
plt.colorbar()
plt.title(r'Unlensed Stack')
plt.subplots()
plt.imshow(lensed_stack-unlensed_stack, extent = extent, origin = 'lower')
plt.colorbar()
plt.title(r'Dipole Stack')
plt.subplots()
plt.plot(bins, kappa_profile, ls = '', marker = 'x')

In [ ]:
gauss = np.exp(np.array(params))
gauss = gauss/max(gauss)

In [ ]:
import scipy
mass_int = np.arange(4, 8, 0.2)
def f(x, a,m,s):
    return a*np.exp(-((x-m)**2)/(2*s**2))




popt, pcov = scipy.optimize.curve_fit(f, mass_int, gauss, p0=[1, 6., 0.2])

In [ ]:
def f2(x,m,s):
    return np.exp(-((x-m)**2)/(2*s**2))


mass_int2 = np.arange(4, 8, 0.02)


plt.plot(mass_int, gauss, color = 'green', ls='',  marker='x', alpha = 0.2)  
plt.plot(mass_int2, f2(mass_int2, popt[1],popt[2]), color = 'green', ls='-', alpha = 0.2)

plt.axvline(6, color = 'green', ls = '--')
plt.xlabel(r'$Mass [M_\odot]$')
plt.ylabel('Normalized Likelihood')

In [ ]:
sample1 = []

for k in range(3):
    sample = np.random.normal(res1[k][1], res1[k][2], 10000)
    sample1.append(sample)

In [ ]:
final1 = []

for k in range(3):
    final = [np.percentile(sample1[k], [16, 50, 84])]
    final1.append(final)

print(final1)

In [ ]:
for k in range(10):
    plt.errorbar(b1[k], d1[k], yerr = np.sqrt(np.diagonal(cov)), ls = '', marker = 'x', color = 'black', alpha = 0.2)
    kappa = lensing.kappa_nfw(mapparams, p1[k], 3, 1, 1100)
    alphaX,_ = lensing.alpha_from_kappa(mapparams, kappa)
    alphaX = alphaX[194:206, 194:206]
    alphaX_cut = []
    for i in range(12):
        alphaX_cut.append(alphaX[i:i+1, :][0])
    alphaX_cut = sum(alphaX_cut)/12
    plt.plot(b1[k], alphaX_cut, color = 'red', alpha = 0.2)    
kappa = lensing.kappa_nfw(mapparams, 2e14, 3, 1, 1100)
alphaX,_ = lensing.alpha_from_kappa(mapparams, kappa)
alphaX = alphaX[194:206, 194:206]
alphaX_cut = []
for i in range(12):
    alphaX_cut.append(alphaX[i:i+1, :][0])
alphaX_cut = sum(alphaX_cut)/12
plt.plot(bins, alphaX_cut, color = 'green', label = 'real')    
plt.legend()